In [1]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import pendulum
from PIL import Image
import pickle

from ipywidgets import IntProgress
from IPython.display import display, clear_output
import IPython

try:
  import google.colab
  from google.colab import drive, files
  runnin_in_colab = True
except:
  runnin_in_colab = False

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

In [2]:
xls_dir="UT_Smoking_Data"
pickles_dir="UT_Smoking_Data_pickles"
x_dir="UT_Smoking_Data_x"
local_full_path="/home/andrei/code/andrei-ka/mocap/notebooks"

categ_colname='Class_label'
ts_field='timestamp_WD' 

In [3]:
raw_ds_index = {
     0:'timestamp_WD', 
     1:'Accelerometer_x_WD', 
     2:'Accelerometer_y_WD', 
     3:'Accelerometer_z_WD', 
     4:'Linear_acceleration_sensor_x_WD', 
     5:'Linear_acceleration_sensor_y_WD', 
     6:'Linear_acceleration_sensor_z_WD', 
     7:'Gyroscope_x_WD', 
     8:'Gyroscope_y_WD', 
     9:'Gyroscope_z_WD',
    10:'Magnetometer_x_WD', 
    11:'Magnetometer_y_WD', 
    12:'Magnetometer_z_WD', 
    13:'Pressure_sensor_WD', 
    14:'Heart_rate_sensor_WD',
    15:'GAP', 
    16:'sevtimestamp_PD', 
    17:'Accelerometer_x_PD', 
    18:'Accelerometer_y_PD',
    19:'Accelerometer_z_PD',
    20:'Linear_acceleration_sensor_x_PD', 
    21:'Linear_acceleration_sensor_y_PD', 
    22:'Linear_acceleration_sensor_z_PD', 
    23:'Gyroscope_x_PD', 
    24:'Gyroscope_y_PD',
    25:'Gyroscope_z_PD', 
    26:'Magnetometer_x_PD', 
    27:'Magnetometer_y_PD', 
    28:'Magnetometer_z_PD', 
    29:'GPS_lat_PD',
    30:'GPS_long_PD', 
    31:'Class_label'}

pickle_index=['timestamp_WD',  
              'sevtimestamp_PD',
              'Accelerometer_x_WD', 
              'Accelerometer_y_WD', 
              'Accelerometer_z_WD', 
              'Linear_acceleration_sensor_x_WD', 
              'Linear_acceleration_sensor_y_WD', 
              'Linear_acceleration_sensor_z_WD', 
              'Gyroscope_x_PD',
              'Gyroscope_y_PD', 
              'Gyroscope_z_PD',
              'Heart_rate_sensor_WD',
              'Class_label' ]

In [4]:
for f_in in os.listdir(pickles_dir): 
    if not (f_in.startswith('Participant') and f_in.endswith('_Data.pickle')) : continue
    print(f_in)

Participant10_Data.pickle
Participant11_Data.pickle
Participant1_Data.pickle
Participant2_Data.pickle
Participant3_Data.pickle
Participant4_Data.pickle
Participant5_Data.pickle
Participant6_Data.pickle
Participant7_Data.pickle
Participant8_Data.pickle
Participant9_Data.pickle


In [5]:
##ok, it's ime to make X, y from a whole dataset...

drop_dups=True
drop_dups_str="_no_dups" if drop_dups else ""

sampling=50  #sampling rate
win_len=20   #wind size in secs

#overlap will add 2nd pass moving init windsow position...
#seems to add only 3% to prec/recall...

overlap=True 
overlap_len=(win_len-10)*sampling  #n secs overlap, dont make it neg !
                                  #here on overap pass we start nead the end of 1st frame
    
overlap_str="_with_overlap" if overlap else ""

feature_set="single" # "accel" or "accel+gyro"

nparts="all"     #test train will be made only by split
ignore_dudes = [ "Participant10" ] #nb always ignore  "Participant10" !!!

if nparts=="8":  #for pure train
  ignore_dudes = [ "Participant10", "Participant3", "Participant7" ] 
if nparts=="2":  #for pure test
  ignore_dudes = [ "Participant10", "Participant11", 
                   "Participant1",  "Participant2", 
                   "Participant4",  "Participant5", 
                   "Participant6",  "Participant8", 
                   "Participant9" ] #nb always ignore  "Participant10" !!!

n_ignored=len(ignore_dudes) - 1

adjusted=1
adjusted_str="_sensor_adjusted" if adjusted != 1 else ""
 
sensors=[ 'Linear_acceleration_sensor_x_WD', 
          'Linear_acceleration_sensor_y_WD', 
          'Linear_acceleration_sensor_z_WD' ] 

sensors_weights={ "Linear_acceleration_sensor_x_WD": 1.0 * adjusted, 
                  "Linear_acceleration_sensor_y_WD": 1.0 * adjusted, 
                  "Linear_acceleration_sensor_z_WD": 1.0 }
    
# our rgb-likes & sensor weights
if feature_set == "accel+gyro" :  
  sensors.extend(['Gyroscope_x_PD', 
                  'Gyroscope_y_PD', 
                  'Gyroscope_z_PD'])

  sensors_weights.update({ "Gyroscope_x_PD": 1.0, 
                           "Gyroscope_y_PD": 1.0, 
                           "Gyroscope_z_PD": 1.0 * adjusted})

if feature_set == "single" : 
    sensors=['Linear_acceleration_sensor_z_WD' ]
    sensors_weights={ 'Linear_acceleration_sensor_z_WD': 1.0 * adjusted }
    
tot_frame_len=sampling*win_len
n_sensors=len(sensors)

#prefer to hardcode expected act_types here...

act_types=[ 'DrinkSD', 'DrinkST', 'Eat', 'Sit', 'SmokeSD', 'SmokeST', 'Stand' ]
n_act_types=len(act_types)
    
remap_acts=True
if remap_acts:
  act_types_maping={ 'drink' : [ 'DrinkSD', 'DrinkST' ],  
                     'smoke' : [ 'SmokeSD', 'SmokeST' ],  
                     'static': [ 'Sit',     'Stand'   ],
                     'eat'   : [ 'Eat'  ] 
                   }
  
id_str=str(len(sensors)) + "apple_sensors_" + str(win_len) + "_win" + overlap_str+drop_dups_str+adjusted_str

x_obj_fname=x_dir + "/" + "x_" + id_str
if n_ignored != 0 : x_obj_fname = x_obj_fname + "_{}ignored".format(n_ignored)
x_obj_fname=x_obj_fname+".dump"

print("f in/out: {}".format(x_obj_fname))

f in/out: UT_Smoking_Data_x/x_1apple_sensors_20_win_with_overlap_no_dups.dump


In [6]:
ts_field='datime_WD'  #cleaned up one, 
                      #otherwise sort will die on timestamp_WD of participant1 

do_plot_activity=False

def plot_activity(activity, data, fname):

    fig, ax = plt.subplots(figsize=(18, 7))
    plt.plot( data[ts_field], data['Accelerometer_x_WD'], color='orchid', linewidth=1.5, label="X-axis")
    plt.plot( data[ts_field], data['Accelerometer_y_WD'], color='dodgerblue', linewidth=1.5,  label="Y-axis")
    plt.plot( data[ts_field], data['Accelerometer_z_WD'], color='dimgrey', linewidth=1.5, label="Z-axis")
    plt.subplots_adjust(hspace=0.2)
    plt.subplots_adjust(top=0.90)
    plt.suptitle(activity)
    plt.legend()
    plt.ylim(-12 , 20)
    plt.grid(True)
    plt.xticks([])
    plt.show()


if do_plot_activity:
    f_ins=[]
    for f_in in os.listdir(pickles_dir):     
        # load raw from pickle    
        if not (f_in.startswith('Participant') and f_in.endswith('_Data.pickle')) : continue

        skip_this_one=False
        for dude in ignore_dudes :
            if f_in.startswith(dude):
                print ("skipping {}".format(dude))
                skip_this_one=True
                break        
        if skip_this_one: continue        
        data = pd.read_pickle(os.path.join(pickles_dir, f_in)) 
        data=data.sort_values(by=ts_field, ignore_index=True)
        data=data2.reset_index(drop=True)
    
        print("{}".format(f_in))
        for activity in np.unique(data[categ_colname]):
          subset = data[data[categ_colname] == activity][2000:3000]
          plot_activity(activity, subset, f_in)   

In [15]:
#someone should revite will cell ! it's so slow...!

ts_field='datime_WD'  #cleaned up one, 
                      #otherwise sort will die on timestamp_WD of participant1 

f_ins=[]
for f_in in os.listdir(pickles_dir):     
    # load raw from pickle    
    if not (f_in.startswith('Participant') and f_in.endswith('_Data.pickle')) : continue
     
    skip_this_one=False
    for dude in ignore_dudes :
        if f_in.startswith(dude):
            print ("skipping {}".format(dude))
            skip_this_one=True
            break        
    if skip_this_one: continue        
    f_ins.append(f_in)

n_files=len(f_ins)
if n_files == 0 :
    raise("no pickles to process!..")


n_participant=0
participants=[]
do_round=False
drop_dups=False

for f_in in f_ins:     
    
    n_files-=1
    print("prcessing {}, {} more left".format(f_in, n_files))

    data = pd.read_pickle(os.path.join(pickles_dir, f_in)) 
       
    act_types_in_file=np.sort(data[categ_colname].unique())
    
    if len(act_types_in_file) != n_act_types :
        
       if n_act_types > len(act_types_in_file) :         
         diff_acts=list(set(act_types) - set(act_types_in_file))
       else:
         diff_acts=list(set(act_types_in_file) - set(act_types)) 
       
       print("achtung, file {} has new or missing activities...".format(f_in, ", ".join(diff_acts))) 
   
    # make windows sets from raw...    
    
    max_count = int(len(data)/tot_frame_len)
    if overlap : max_count+=max_count
        
    print("expecting ~{} images in total...".format(max_count))
    
    pbar = IntProgress(min=0, max=max_count) 
    display(pbar) 
    
    a_stat={}
    for act_type in act_types :
      data_cursor = data[(data[categ_colname] == act_type)].copy() 
      data_act_type_len=len(data_cursor)-tot_frame_len
      a_stat[act_type]=[]
      
      data2=data_cursor.sort_values(by=ts_field, ignore_index=True)
      data_cursor=data2.reset_index(drop=True)
    
      a_stat[act_type]=len(data_cursor)   
      
      if drop_dups: 
        #report first
        df=data_cursor.copy()
        duplicateRows = df[df.duplicated()]
        ndups=len(duplicateRows)
        pcntage=int( 100*float(ndups)/float(len(df)) ) 
        a_stat["dups"]=ndups 
        a_stat["dups_pcnt"]=pcntage 
        #print("\n{} ({}) dups in act type {}".format(ndups, pcntage, act_type))
        
        data_cursor.drop_duplicates(inplace=True)
        duplicateRows = data_cursor[data_cursor.duplicated()]
        ndups=len(duplicateRows)
        #print("ndups afer cleaning {}\n".format(ndups)) 
        
       
      
    stat_dict={ "name": f_in, "activities": a_stat}
    participants.append(stat_dict)
    
    print("done with participant {}\n".format(n_participant))
    n_participant=n_participant+1

print("all done..")

skipping Participant10
prcessing Participant11_Data.pickle, 9 more left
expecting ~784 images in total...


IntProgress(value=0, max=784)

done with participant 0

prcessing Participant1_Data.pickle, 8 more left
expecting ~1808 images in total...


IntProgress(value=0, max=1808)

done with participant 1

prcessing Participant2_Data.pickle, 7 more left
expecting ~1978 images in total...


...

done with participant 7

prcessing Participant8_Data.pickle, 1 more left
expecting ~854 images in total...


IntProgress(value=0, max=854)

done with participant 8

prcessing Participant9_Data.pickle, 0 more left
expecting ~1022 images in total...


IntProgress(value=0, max=1022)

done with participant 9

all done..


In [16]:
participants

[{'name': 'Participant11_Data.pickle',
  'activities': {'DrinkSD': 56000,
   'DrinkST': 56000,
   'Eat': 56000,
   'Sit': 56000,
   'SmokeSD': 56000,
   'SmokeST': 56000,
   'Stand': 56000}},
 {'name': 'Participant1_Data.pickle',
  'activities': {'DrinkSD': 129200,
   'DrinkST': 129200,
   'Eat': 129200,
   'Sit': 129200,
   'SmokeSD': 129200,
   'SmokeST': 129200,
   'Stand': 129200}},
 {'name': 'Participant2_Data.pickle',
  'activities': {'DrinkSD': 141400,
   'DrinkST': 141400,
   'Eat': 141400,
   'Sit': 141400,
   'SmokeSD': 141400,
   'SmokeST': 141400,
   'Stand': 141400}},
 {'name': 'Participant3_Data.pickle',
  'activities': {'DrinkSD': 145000,
   'DrinkST': 145000,
   'Eat': 145000,
   'Sit': 145000,
   'SmokeSD': 145000,
   'SmokeST': 145000,
   'Stand': 145000}},
 {'name': 'Participant4_Data.pickle',
  'activities': {'DrinkSD': 111300,
   'DrinkST': 111300,
   'Eat': 111300,
   'Sit': 111300,
   'SmokeSD': 111300,
   'SmokeST': 111300,
   'Stand': 111300}},
 {'name': 'Parti

In [ ]:
#pip install pq-tool
#os.environ['x_dir_to_list'] = local_full_path+"/"+x_dir
#!cd $x_dir_to_list && ls -l && for x in *; do pq keys $x; done